# Projeto : Data Augmentation com o dataset  MNIST

---


O projeto consiste em realizar o processo de translação  de imagens para observar a performance do modelo de aprendizado de máquina e comparar com o notebook exemplo que foi dado.

In [ ]:
# Common imports
import numpy as np
import pandas as pd 
import os


# Para ajudar na reproducibilidade
# Aqui a seed é 42, mas não há nada de especial nisso
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

def save_fig(fig_id, tight_layout=True):
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(fig_id, format='png', dpi=600)

Abaixo está o código para importação do conjunto de dados.

In [ ]:
#Importando dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8) # fetch_openml() returns targets as strings

In [ ]:
X, y = mnist["data"], mnist["target"]
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


In [ ]:
#funcao para plotar um digito do MNIST
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

In [ ]:
#funcao para plotar varios digitos do MNIST
def plot_digits(data):
  for x in data:
    image = x.reshape(28, 28)
    plt.figure()
    plt.imshow(x, cmap = mpl.cm.binary,interpolation="nearest")
    plt.axis("off")

Abaixo está a função para translacionar os dados. Foi utilizando a biblioteca openCV.

In [ ]:
import cv2 

def translate_images(features,label):
    lista_coordenadas = [(1,0), (-1,0), (0,1), (0,-1)]
    lista_imagens = []
    lista_rotulo = []
    for i in range(len(features)):
      rotulo = label[i]
      for c in lista_coordenadas:
        lista_imagens.append(cv2.warpAffine(features[i].reshape(28,28), np.float32([[1, 0, c[0]], [0, 1, c[1]]]) , (28, 28)).reshape(-1))
        lista_rotulo.append(rotulo)
    lista_imagens = np.array(lista_imagens)
    lista_rotulo  = np.array(lista_rotulo)
    return lista_imagens,lista_rotulo

In [ ]:
x_train, x_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:] #dividindo os dados para treino e teste

In [ ]:
x_trans,y_trans = translate_images(x_train,y_train)         #usando a translação



print('Quantidade de dados com translacao:',len(x_trans))

Quantidade de dados com translacao: 240000


In [ ]:
shuffle_index = np.random.permutation(len(x_trans))             #embaralhando os dados
x_train, y_train = x_trans[shuffle_index], y_trans[shuffle_index]

Abaixo está o grid search com todos os parâmetros utilizados no notebook de exemplo. Não quero enviesar os dados colocando mais parâmetros, visto que o objetivo é comparar os erros de generalização.

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([
                 ('std_scaler', StandardScaler()),
                 ('estimator',SGDClassifier(max_iter =10,random_state = 42))
])

param_grid = [
              {'estimator__loss' : ['hinge','log'],
               'estimator__alpha' : [1e-4, 1e-2,1],
              } ]

grid_search = GridSearchCV(pipe,param_grid, cv =5) #obs : tive que retirar o verbose e n_jobs pois estava dando erro

grid_search.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_mo

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('std_scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('estimator',
                                        SGDClassifier(alpha=0.0001,
                                                      average=False,
                                                      class_weight=None,
                                                      early_stopping=False,
                                                      epsilon=0.1, eta0=0.0,
                                                      fit_intercept=True,
                                                      l1_ratio=0.15,
                                                      learning_rate='optimal',
                  

Observando os scores relacionados com cada combinação de parâmetros acima. Observe que os scores estão um pouco abaixo em relação ao notebook.

In [ ]:
results = pd.concat([pd.DataFrame(grid_search.cv_results_["params"]),
                     pd.DataFrame(grid_search.cv_results_['std_test_score'], 
                                  columns=["Std"]),
                     pd.DataFrame(grid_search.cv_results_["mean_test_score"], 
                                  columns=["Score"])],axis=1)

results.sort_values("Score", ascending=False) #Ordenamento decrescente

,estimator__alpha,estimator__loss,Std,Score
0,0.0001,hinge,0.001735,0.872075
1,0.0001,log,0.002023,0.872008
2,0.0100,hinge,0.001165,0.853933
3,0.0100,log,0.001163,0.853200
4,1.0000,hinge,0.004709,0.810733
5,1.0000,log,0.004505,0.785729


In [ ]:

modelo = Pipeline([
                   ('std_scaler', StandardScaler()),
                   ('estimator',SGDClassifier(max_iter=10,random_state=42))
])


modelo.set_params(**grid_search.best_params_)     #colocando os melhores parâmetros

Pipeline(memory=None,
         steps=[('std_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('estimator',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=10, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=42,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_start=False))],
         verbose=False)

In [ ]:
modelo.fit(x_train,y_train)    #treinando o modelo


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


Pipeline(memory=None,
         steps=[('std_scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('estimator',
                 SGDClassifier(alpha=0.0001, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                               max_iter=10, n_iter_no_change=5, n_jobs=None,
                               penalty='l2', power_t=0.5, random_state=42,
                               shuffle=True, tol=0.001, validation_fraction=0.1,
                               verbose=0, warm_start=False))],
         verbose=False)

In [ ]:
modelo.classes_ #observando as clases

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int8)

In [ ]:
from sklearn.multiclass import OneVsOneClassifier #utilizando a técnica One Vs One para classificação multiclasse

modelo = Pipeline([
                   ('std_scaler',StandardScaler()),
                   ('estimator',SGDClassifier(max_iter=1000,random_state = 42)) #aumentando o numero de iteracoes
])

modelo.set_params(**grid_search.best_params_)

ovo_clf = OneVsOneClassifier(modelo)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(ovo_clf,x_train,y_train, cv =5)  #score amostral

Abaixo observe que o score amostral diminuiu um pouco em relação ao notebook do exemplo.

In [ ]:
scores.mean()  #média do score

0.8997166666666667

In [ ]:
ovo_clf.fit(x_train, y_train) #treinando para estimar o erro de generalizacao

len(ovo_clf.estimators_)

45

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = ovo_clf.predict(x_test)

conf_matrix = confusion_matrix(y_test,y_pred)

conf_matrix #plotando a matriz de confusao

array([[ 966,    1,    2,    1,    1,    3,    2,    2,    2,    0],
       [   0, 1123,    4,    2,    1,    1,    3,    0,    1,    0],
       [   7,    6,  934,   23,   18,    2,   11,    9,   21,    1],
       [   3,    3,   13,  928,    1,   21,    3,   13,   21,    4],
       [   1,    1,    4,    1,  938,    0,    6,    4,    1,   26],
       [  10,    4,   10,   46,    8,  779,   12,    5,   17,    1],
       [  13,    3,    9,    1,    5,   11,  914,    1,    1,    0],
       [   1,    3,   17,    6,    7,    0,    1,  967,    1,   25],
       [   7,    9,    7,   17,   12,   19,    7,    9,  880,    7],
       [   5,    8,    1,   11,   33,    3,    0,   24,    4,  920]])

Observe abaixo que o score de generalização ficou um pouco maior que o score de generalização do notebook anterior.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred) #erro de generalização

0.9349

# Considerações finais

---- 

Curiosamente o modelo com o processo de data augmentation mesmo não conseguindo um score amostral mais alto, conseguiu um score de generalização um pouco mais alto que o notebook de exemplo. 